# Preamble

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%matplotlib inline

In [3]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

import numpy as np
import pickle

from multiprocessing import Pool 
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from tqdm import tqdm_notebook as tqdm


import integrals as integ
import diff_eq_multi_2 as diff_eq
import physics as phys

# Load the standard solution.
a = pickle.load(open('../data/std_soln_He.p','rb'))

std_Tm = interp1d(a[0,:], a[1,:])
std_xH = interp1d(a[0,:], a[2,:])

# Constraints on $Q_\mathrm{eff}$

In [4]:
def max_Q_d_eff(m_m, m_C, f, Q):
    
    g_em    = np.sqrt(4*np.pi*phys.alpha)
    mu_pC   = m_C*phys.mp/(m_C + phys.mp)
    f_ratio = 1. + phys.omega_baryon/(f*phys.omega_DM)
    
    max_SI             = 0.26*(m_C/1e9)**(3/8) * np.sqrt(np.sqrt(4*np.pi)/g_em)
    max_CMB            = 1e-3 * np.sqrt(m_C/1e9) * np.sqrt(m_m/1e9) * (1e9/(m_m + m_C))**0.25 * f_ratio**0.25
    max_tight_coupling = 0.08 * np.sqrt(Q) * np.sqrt(m_C/1e8) * (1e9/m_m)**0.25
    
    return np.min((max_SI, max_CMB, max_tight_coupling))

# Fixed Parameters

In [5]:
rs_start = 4000
rs_vec   = np.flipud(np.arange(10., rs_start, 1))

# Fixed Parameters
f   = 1e-4
m_C = 1e8   

# Loop

In [12]:
# Loop Points
m_m_arr = 10**np.arange(6.2, 13.2, 0.2)
Q_arr   = 10**np.arange(-4.,  0.2, 0.2) # 21 entries



soln_arr  = np.zeros((m_m_arr.size, Q_arr.size, rs_vec.size, 6))
error_arr = [] 

for i,m_m in enumerate(tqdm(m_m_arr)):
    
    # Initial conditions are T_CMB - T_b, T_b - T_m, log(T_C), log(V_bm), log(V_cb), xe
    init_cond = [1e-6, 1e-6, np.log(1e-6), np.log(1e-16), np.log(29e5/phys.c), 1.] 
    
    def solve_ODE_with_Q(params):
        
        Q       = params[0]
        Q_d_eff = params[1]
        
        def to_integrate(rs, var):

            neutral_H  = True
            neutral_He = True

            return diff_eq.DM_baryon_ODE(
                rs, var,
                m_m, m_C, Q, Q_d_eff, neutral_H, neutral_He, f, True 
            )

        # Number of attempts. 
        N_attempts = 0
        solved     = False
        rtol       = 1e-4
        max_step   = 100.

        while (not solved) and (N_attempts < 10):

            try:

                N_attempts += 1

                soln = solve_ivp(
                    to_integrate, [rs_vec[0], rs_vec[-1]], init_cond, 
                    method='BDF', t_eval=rs_vec, 
                    rtol=rtol,
                )

            except:
                # solve_ivp returned an error. Adjust the tolerance
                # and step size and try again. 

                rtol /= 1.2
                max_step /= 1.5

            else:
                # solve_ivp completed successfully. 
                solved = True

        return soln 
    
    # Effective dark sector charge, maximum value fro constraints.
    Q_d_eff_arr = np.array([max_Q_d_eff(m_m, m_C, f, Q) for Q in Q_arr])
    
    # Temporary array for solution. 
    soln = np.zeros((rs_vec.size, 6))
    
    # Parallelization.
    
    solve_ODE_with_Q([Q_arr[0], Q_d_eff_arr[0]])
#     pool = Pool(21)
    
#     soln_raw = list(tqdm(pool.imap(solve_ODE_with_Q, zip(Q_arr, Q_d_eff_arr))))

0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
8.333333333333334e-05
0.0001


Process ForkPoolWorker-227:
Process ForkPoolWorker-226:
Process ForkPoolWorker-224:


8.333333333333334e-05
6.944444444444446e-05


Process ForkPoolWorker-217:
Process ForkPoolWorker-215:
Process ForkPoolWorker-213:
Process ForkPoolWorker-211:
Process ForkPoolWorker-212:
Process ForkPoolWorker-216:
Process ForkPoolWorker-218:
Process ForkPoolWorker-214:
Process ForkPoolWorker-210:
Process ForkPoolWorker-228:
Process ForkPoolWorker-225:
Process ForkPoolWorker-223:
Process ForkPoolWorker-222:
Process ForkPoolWorker-221:


5.7870370370370386e-05


...

1.93806699467815e-05


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 246, in _bootstrap
    util._close_stdin()
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hongwan/anaconda/envs/pyth

2.3256803936137797e-05


KeyboardInterrupt
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
Process ForkPoolWorker-266:
Process ForkPoolWorker-267:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/pool.py", line 108, in

  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/multiprocessing